**SITI MA'RIFATUL KHOIRIYAH_210411100007**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/psd/AUDIO TORONTO

Mounted at /content/drive
/content/drive/MyDrive/psd/AUDIO TORONTO


In [ ]:
%%capture
pip install librosa

# **EKSTRAKSI FITUR**

In [ ]:
# Import Library
import os
import librosa
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis, mode

In [ ]:
folders=['YAF_sad','YAF_pleasant_surprised','YAF_neutral',
         'YAF_happy','YAF_fear','YAF_disgust','YAF_angry',
         'OAF_Sad','OAF_Pleasant_surprise','OAF_neutral',
         'OAF_happy','OAF_Fear','OAF_disgust',
         'OAF_angry',
         ]

In [ ]:
def calculate_statistics(audio_path):
    y, sr = librosa.load(audio_path)

    # UNTUK MENGHITUNG NILAI STATISTIKA
    mean = np.mean(y)
    std_dev = np.std(y)
    max_value = np.max(y)
    min_value = np.min(y)
    median = np.median(y)
    skewness = skew(y)  # Calculate skewness
    kurt = kurtosis(y)  # Calculate kurtosis

    # UNTUK MENGHITUNG NILAI ZCR
    zcr_mean = np.mean(librosa.feature.zero_crossing_rate(y=y))
    zcr_median = np.median(librosa.feature.zero_crossing_rate(y=y))
    zcr_std_dev = np.std(librosa.feature.zero_crossing_rate(y=y))
    zcr_kurtosis = kurtosis(librosa.feature.zero_crossing_rate(y=y)[0])
    zcr_skew = skew(librosa.feature.zero_crossing_rate(y=y)[0])

    # UNTUK MENGHITUNG NILAI RMSE
    rmse = np.sum(y**2) / len(y)
    rmse_median = np.median(y**2)
    rmse_std_dev = np.std(y**2)
    rmse_kurtosis = kurtosis(y**2)
    rmse_skew = skew(y**2)

    return [zcr_mean, zcr_median, zcr_std_dev, zcr_kurtosis, zcr_skew, rmse, rmse_median, rmse_std_dev, rmse_kurtosis, rmse_skew]

In [ ]:
# Membuat DataFrame untuk menyimpan hasil
data = []

In [ ]:
# Loop melalui setiap folder dan file audio
for folder in folders:
    folder_path = f'{folder}'
    for filename in os.listdir(folder_path):
        if filename.endswith('.wav'):
            audio_path = os.path.join(folder_path, filename)
            statistics = calculate_statistics(audio_path)
            data.append([folder, filename] + statistics)

In [ ]:
# Membuat DataFrame dari data
columns =  ['Label', 'File'] + ['ZCR Mean', 'ZCR Median', 'ZCR Std Dev', 'ZCR Kurtosis', 'ZCR Skew', 'RMSE', 'RMSE Median', 'RMSE Std Dev', 'RMSE Kurtosis', 'RMSE Skew']
df = pd.DataFrame(data, columns=columns)

In [ ]:
# Menampilkan file CSV
df

,Label,File,ZCR Mean,ZCR Median,ZCR Std Dev,ZCR Kurtosis,ZCR Skew,RMSE,RMSE Median,RMSE Std Dev,RMSE Kurtosis,RMSE Skew
0,YAF_sad,YAF_gas_sad.wav,0.192857,0.042480,0.265538,0.054455,1.328582,0.000980,0.000194,0.002083,24.307945,4.267333
1,YAF_sad,YAF_goose_sad.wav,0.182617,0.046875,0.245189,0.037876,1.320627,0.002526,0.000275,0.005935,28.021587,4.589746
2,YAF_sad,YAF_chair_sad.wav,0.133214,0.021484,0.202694,2.302072,1.881587,0.000576,0.000161,0.001048,14.417235,3.420924
3,YAF_sad,YAF_germ_sad.wav,0.134588,0.037109,0.216944,3.035306,2.075205,0.001716,0.000320,0.003778,23.784036,4.284389
4,YAF_sad,YAF_limb_sad.wav,0.101898,0.033203,0.187808,5.820668,2.693270,0.005470,0.000982,0.011635,34.287124,4.722479
...,...,...,...,...,...,...,...,...,...,...,...,...
2795,OAF_angry,OAF_lot_angry.wav,0.088930,0.055908,0.103065,6.091986,2.631815,0.002318,0.000237,0.006366,39.610165,5.619980
2796,OAF_angry,OAF_thin_angry.wav,0.122681,0.060303,0.121685,0.990843,1.453864,0.004285,0.000189,0.011142,22.885030,4.356848
2797,OAF_angry,OAF_perch_angry.wav,0.081423,0.060547,0.083024,5.690889,2.393663,0.003017,0.000130,0.008586,47.058684,5.896208
2798,OAF_angry,OAF_luck_angry.wav,0.083126,0.062744,0.086203,3.685553,1.973698,0.000502,0.000051,0.001268,30.544881,4.824743


In [ ]:
# Menyimpan DataFrame ke dalam file CSV
df.to_csv('hasil_statistik_audio.csv', index=False)

# **PRE-PROCESSING NORMALISASI dengan Z-SCORE**

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Baca file CSV
norm = pd.read_csv("hasil_statistik_audio.csv")

In [ ]:
# Tentukan kolom yang akan distandarisasi
kolom = ['ZCR Mean', 'ZCR Median', 'ZCR Std Dev', 'ZCR Kurtosis', 'ZCR Skew', 'RMSE', 'RMSE Median', 'RMSE Std Dev', 'RMSE Kurtosis', 'RMSE Skew']


SIMPAN MODEL NORMALISASI DAN MODEL KLASIFIKASI

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from pickle import dump

In [ ]:
# Baca data dari file CSV
dataknn= pd.read_csv('hasil_statistik_audio.csv')
# Pisahkan fitur (X) dan label (y)
X = dataknn.drop(['Label','File'], axis=1)  # Ganti 'target_column' dengan nama kolom target
y = dataknn['Label']
# split data into train and test sets
X_train,X_test,y_train, y_test= train_test_split(X, y, random_state=1, test_size=0.2)
# define scaler
scaler = StandardScaler()
# fit scaler on the training dataset
scaler.fit(X_train)
# save the scaler
#dump(scaler, open('scaler.pkl', 'wb'))
# transform the training dataset
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
import pickle
with open('scaler.pkl', 'rb') as standarisasi:
    loadscal= pickle.load(standarisasi)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
K = 50
acc = np.zeros((K-1))

for n in range(1,K,2):
    knn = KNeighborsClassifier(n_neighbors= n, metric = "euclidean").fit(X_train_scaled, y_train)
    y_pred = knn.predict(X_test_scaled)

    acc[n-1]= accuracy_score(y_test,y_pred)

print('akurasi terbaik adalah ', acc.max(), 'dengan nilai k =', acc.argmax()+1)

akurasi terbaik adalah  0.7089285714285715 dengan nilai k = 7


In [ ]:
knn = KNeighborsClassifier(n_neighbors= 11, metric = "euclidean")
dump(knn, open('modelknn.pkl', 'wb'))


In [ ]:
import pickle
with open('modelknn.pkl', 'rb') as knn:
    loadknn= pickle.load(knn)
loadknn.fit(X_train_scaled, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=11)

In [ ]:
accuracy = accuracy_score(y_test,y_pred)
print("Akurasi:",accuracy)

Akurasi: 0.6464285714285715


# **REDUKSI DATA DENGAN PCA**

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=9)
pca.fit(X_train_scaled)
pca_train = pca.transform(X_train_scaled)

In [ ]:
pca_train.shape

(2240, 9)

In [ ]:
pca_test = pca.transform(X_test_scaled)

In [ ]:
type(pca_test)

numpy.ndarray

In [ ]:
pca_test.shape

(560, 9)

In [ ]:
dump(pca_train, open('PCA9.pkl', 'wb'))

In [ ]:
import pickle
with open('PCA9.pkl', 'rb') as pca:
    loadpca= pickle.load(pca)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=11)
classifier.fit(pca_train, y_train)

KNeighborsClassifier(n_neighbors=11)

In [ ]:
y_prediksi = classifier.predict(pca_test)
y_prediksi

array(['OAF_Fear', 'YAF_disgust', 'YAF_neutral', 'YAF_neutral',
       'OAF_neutral', 'OAF_neutral', 'YAF_neutral', 'OAF_angry',
       'OAF_happy', 'OAF_neutral', 'YAF_sad', 'YAF_neutral', 'OAF_angry',
       'OAF_angry', 'OAF_Pleasant_surprise', 'OAF_angry', 'OAF_Sad',
       'YAF_sad', 'OAF_angry', 'YAF_sad', 'YAF_pleasant_surprised',
       'OAF_neutral', 'OAF_angry', 'YAF_fear', 'OAF_neutral', 'YAF_sad',
       'YAF_angry', 'YAF_fear', 'OAF_Fear', 'OAF_happy', 'OAF_neutral',
       'OAF_Fear', 'YAF_neutral', 'YAF_angry', 'YAF_sad', 'YAF_angry',
       'OAF_Fear', 'OAF_disgust', 'YAF_happy', 'OAF_disgust',
       'YAF_disgust', 'OAF_Sad', 'YAF_neutral', 'YAF_fear', 'OAF_happy',
       'OAF_happy', 'YAF_happy', 'YAF_pleasant_surprised', 'OAF_neutral',
       'OAF_Sad', 'OAF_Pleasant_surprise', 'OAF_happy', 'OAF_Sad',
       'YAF_pleasant_surprised', 'YAF_disgust', 'OAF_disgust',
       'OAF_angry', 'OAF_disgust', 'YAF_disgust', 'OAF_Sad', 'OAF_happy',
       'OAF_Fear', 'OAF_Sad', '

In [ ]:
acc_pca= accuracy_score(y_test,y_prediksi)
print("Akurasi:",acc_pca)
#Akurasi: 0.7153024911032029

Akurasi: 0.6821428571428572
